_Autor:_    __Jesús Casado__ <br> _Revisión:_ __21/01/2019__ <br>


__Descripción__:<br>
Se extraen las series de las estaciones meteorológicas y de aforo necesarias para la simulación de la cuenca del río Nansa.


__Cosas a corregir__ <br>


__Índice__<br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
%matplotlib inline
from simpledbf import Dbf5
import os
from datetime import datetime, timedelta

## 1. Estaciones de Cantabria

In [2]:
rutaSAIH = 'F:/Series/CHC/SAI/ESTACIONES CHC DE CANTABRIA/'
rutaSAIH1 = 'F:/Series/CHC/SAI/ESTACIONES CHC DE CANTABRIA/Hasta junio de 2015/Cincominutales hasta junio de 2015/'
rutaSAIH2 = 'F:/Series/CHC/SAI/ESTACIONES CHC DE CANTABRIA/Desde julio de 2015/Cincominutales desde julio de 2015/'

In [3]:
SAIH_CHC_diario('1252', rutaSAIH, verbose=True)

NameError: name 'SAIH_CHC_diario' is not defined

In [ ]:
def SAIH_CHC_diario(estacion, ruta, verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion: str o int. Nombre de la estación
    ruta:     str
    verbose:  boolean.
    
    Salidas:
    --------
    """
    
    # rutas
    rutaSAIH1 = ruta + '/Hasta junio de 2015/'
    rutaSAIH2 = ruta + '/Desde julio de 2015/'
    
    # PARTE 1
    # -------
    # encontrar archivos de la estación
    folders = os.listdir(rutaSAIH1)
    for f, folder in enumerate(folders):
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH1 + folder + '/'
            files = []
            for file in os.listdir(ruta):
                if file[:4] == folder[:4]:
                    files.append(file)

            # Importar datos cincominutales
            data1 = pd.DataFrame()
            uds = {}
            for i, file in enumerate(files):
                # importar serie original
                aux = pd.read_csv(ruta + file, sep=';', encoding='latin-1', decimal=',', low_memory=False)
                aux.dropna(axis=0, how='all', inplace=True)
                aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
                aux.set_index('Fecha', drop=True, inplace=True)
                # reordenar 'aux' por señales
                signals = aux['Nombre señal'].unique()
                aux2 = pd.DataFrame()
                for signal in signals:
                    temp = aux.loc[aux['Nombre señal'] == signal, ['Valor', 'Unidades', 'Calidad']]
                    uds[signal] = temp.Unidades[0]
                    temp.columns = [signal, 'Unidades', 'Calidad']
                    aux2 = pd.concat((aux2, temp[signal]), axis=1)
                # concatenar a la serie generada
                data1 = pd.concat((data1, aux2), axis=0)

            # Agregar datos a diarios
            data1_d = data1.groupby(by=[data1.index.year, data1.index.month, data1.index.day]).mean()
            data1_d.index = [pd.datetime(y, m, d).date() for (y, m, d) in data1_d.index]

            st, en = data1_d.index[0], data1_d.index[-1]
            if data1_d.shape[0] != pd.date_range(st, en).shape[0]:
                aux = data1_d.copy()
                data1_d = pd.DataFrame(index=pd.date_range(st, en), columns=data1_d.columns)
                for date in aux.index:
                    data1_d.loc[date, :] = aux.loc[date, :]
            del files
            
    # PARTE 2
    # -------
    # encontrar carpeta de la estación
    folders = os.listdir(rutaSAIH2)
    for folder in folders:
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH2 + folder + '/'
            # encontrar archivos de la estación
            files = os.listdir(ruta)
            
            # Importar datos cincominutales
            data2 = pd.DataFrame()
            for file in files:
                aux = pd.read_csv(ruta + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
                aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
                aux.set_index('Fecha/Hora', drop=True, inplace=True)
                aux.index.name = 'Fecha'
                data2 = pd.concat((data2, aux), axis=0)

            # Agregar datos a diarios
            data2_d = data2.groupby(by=[data2.index.year, data2.index.month, data2.index.day]).mean()
            data2_d.index = [pd.datetime(y, m, d).date() for (y, m, d) in data2_d.index]

            st, en = data2_d.index[0], data2_d.index[-1]
            if data2_d.shape[0] != pd.date_range(st, en).shape[0]:
                print('¡ERROR EN PARTE 2! Menor número de filas que días entre la fecha inicial y final.')
        
    # UNIR SERIES
    # -----------
    # unir las dos series
    if ('data1_d' in locals()) and ('data2_d' in locals()):
        st1, en1 = data1_d.index[0], data1_d.index[-1]
        st2, en2 = data2_d.index[0], data2_d.index[-1]
        idx = pd.date_range(st1, en2)
        data_d = pd.DataFrame(index=idx, columns=data1_d.columns)
        data_d.loc[st2:en2,:] = data2_d
        data_d.loc[st1:en1,:] = data1_d
    elif ('data1_d' in locals()) and ('data2_d' not in locals()):
        data_d = data1_d.copy()
    elif ('data2_d' in locals()) and ('data1_d' not in locals()): 
        data_d = data2_d.copy()
    # corregir columnas
    cols = []
    for col in data_d.columns:
        if col[-8:] == 'XACQRIO1':
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('caudal_m³/s')
        elif col[-8:] == 'XAINRIO1':
            data_d[col] = [round(d, 3) for d in data_d[col]]
            cols.append('nivel_m')
        elif col[-8:] == 'XAIPCINC':
            data_d[col] = data_d[col] * 24 * 60 / 5 # precipitación diaria acumulada
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('precipitacion_mm')
        elif col[-8:] == 'XAITEMEX':
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('temperatura_ºC')
    data_d.columns = cols
    data_d.index.name = 'Fecha'

    # comprobar fechas
    st, en = data_d.index[0], data_d.index[-1]
    if data_d.shape[0] != pd.date_range(st, en).shape[0]:
        print('¡ERROR AL UNIR SERIES! Distinto número de filas que días entre la fecha inicial y final.')
    
    # exportar
    if not os.path.exists(rutaSAIH + '1d/'):
            os.makedirs(rutaSAIH + '1d/')
    data_d.to_csv(rutaSAIH + '1d/' + folder + '.csv', sep=',', na_rep=-100)
    
    if verbose == True:
        print('nº de días de las serie:\t', data_d.shape[0])
        print('variables:\t', list(data_d.columns))

In [ ]:
# encontrar estaciones
stns = [file[-4:] for file in os.listdir(rutaSAIH1)]
for file in os.listdir(rutaSAIH2):
    if file[-4:] not in stns:
        stns.append(file[-4:])
stns.sort()

len(stns)

In [ ]:
for i, stn in enumerate(stns):
    print('estación', str(stn), '\tserie', i, 'de', len(stns))
    SAIH_CHC_diario(stn, rutaSAIH)
    print()

## 2. Estaciones fuera de Cantabria

In [ ]:
rutaSAIH = 'F:/Series/CHC/SAI/0_IHCantabria_20180626/'

__Crear carpetas antes de 2015__

In [ ]:
ruta1 = rutaSAIH + 'Hasta junio de 2015/'
os.chdir(ruta)

# encontrar archivos csv
files = []
for file in os.listdir():
    if file[-3:] == 'csv':
        files.append(file)

# Crear carpeta de cada estación y mover archivos a la carpeta correspondiente
stns1 = []
for file in files:
    stn = file[:4]
    if stn not in stns1:
        stns1.append(stn)
        if not os.path.exists(stn):
            os.makedirs(stn)
    os.rename(ruta + file, ruta + '/' + stn + '/' + file)

__Crear carpetas desde 2015__

In [ ]:
ruta2 = rutaSAIH + 'Desde julio de 2015/'
os.chdir(ruta)

# encontrar archivos csv
files = []
for file in os.listdir():
    if file[-3:] == 'csv':
        files.append(file)

# Crear carpeta de cada estación y mover archivos a la carpeta correspondiente
stns2 = []
for file in files:
    stn = file[8:12]
    if stn not in stns2:
        stns2.append(stn)
        if not os.path.exists(stn):
            os.makedirs(stn)
    os.rename(ruta + file, ruta + '/' + stn + '/' + file)

In [ ]:
stns1 = os.listdir(ruta1)
stns2 = os.listdir(ruta2)
stns = list(set(stns1 + stns2))
stns.sort()
len(stns)

## Función

In [ ]:
def SAIH_CHC_diario2(estacion, ruta, agregacion='1d', verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion:   str o int. Nombre de la estación
    ruta:       str.
    agregacion: str. Resolución temporal a la que agregar los datos: '1d' diaria, '1h' horaria, None datos originales cincominutales
    verbose:    boolean.
    
    Salidas:
    --------
    """

    # rutas
    rutaSAIH1 = ruta + '/Hasta junio de 2015/'
    rutaSAIH2 = ruta + '/Desde julio de 2015/'

    signames = ['AIPCINC', 'ACQRIO1', 'AINRIO1', 'AITEMEX']

    # PARTE 1
    # -------
    # encontrar archivos de la estación
    folders = os.listdir(rutaSAIH1)
    for f, folder in enumerate(folders):
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH1 + folder + '/'
            files = []
            for file in os.listdir(ruta):
                if file[:4] == folder[:4]:
                    files.append(file)

            # Importar datos cincominutales
            data1 = pd.DataFrame()
            for i, file in enumerate(files):
                # importar serie original
                aux = pd.read_csv(ruta + file, sep=';', encoding='latin-1', decimal=',', low_memory=False)
                aux.dropna(axis=0, how='all', inplace=True)
                aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
                aux.set_index('Fecha', drop=True, inplace=True)
                # reordenar 'aux' por señales
                signals, cols = [], []
                for signal in aux['Nombre señal'].unique():
                    if signal[-7:] in signames:
                        signals.append(signal)
                aux2 = pd.DataFrame(columns=cols)
                for signal in signals:
                    temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
                    cols = list(aux2.columns)
                    aux2 = pd.concat((aux2, temp), axis=1)
                    aux2.columns = cols + [str(estacion) + 'X' + signal[-7:]]
                # concatenar a la serie generada
                data1 = pd.concat((data1, aux2), axis=0)

            # Agregar datos a diarios
            data1_d = data1.groupby(by=[data1.index.year, data1.index.month, data1.index.day]).mean()
            data1_d.index = [pd.datetime(y, m, d).date() for (y, m, d) in data1_d.index]

            st, en = data1_d.index[0], data1_d.index[-1]
            if data1_d.shape[0] != pd.date_range(st, en).shape[0]:
                aux = data1_d.copy()
                data1_d = pd.DataFrame(index=pd.date_range(st, en), columns=data1_d.columns)
                for date in aux.index:
                    data1_d.loc[date, :] = aux.loc[date, :]
            del files, folders
            break

    # PARTE 2
    # -------
    # encontrar carpeta de la estación
    folders = os.listdir(rutaSAIH2)
    for folder in folders:
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH2 + folder + '/'
            # encontrar archivos de la estación
            files = os.listdir(ruta)

            # Importar datos cincominutales
            data2 = pd.DataFrame()
            for file in files:
                aux = pd.read_csv(ruta + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
                aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
                aux.set_index('Fecha/Hora', drop=True, inplace=True)
                aux.index.name = 'Fecha'
                cols = []
                for col in aux.columns:
                    if col[-7:] in signames:
                        cols.append(col)
                aux = aux.loc[:, cols]
                data2 = pd.concat((data2, aux), axis=0)

            # Agregar datos a diarios
            data2_d = data2.groupby(by=[data2.index.year, data2.index.month, data2.index.day]).mean()
            data2_d.index = [pd.datetime(y, m, d).date() for (y, m, d) in data2_d.index]

            st, en = data2_d.index[0], data2_d.index[-1]
            if data2_d.shape[0] != pd.date_range(st, en).shape[0]:
                aux = data2_d.copy()
                data2_d = pd.DataFrame(index=pd.date_range(st, en), columns=data2_d.columns)
                for date in aux.index:
                    data2_d.loc[date, :] = aux.loc[date, :]
            del files, folders
            break

    # UNIR SERIES
    # -----------
    # unir las dos series
    if ('data1_d' in locals()) and ('data2_d' in locals()):
        st1, en1 = data1_d.index[0], data1_d.index[-1]
        st2, en2 = data2_d.index[0], data2_d.index[-1]
        idx = pd.date_range(st1, en2)
        data_d = pd.DataFrame(index=idx, columns=data1_d.columns)
        data_d.loc[st2:en2,:] = data2_d
        data_d.loc[st1:en1,:] = data1_d
    elif ('data1_d' in locals()) and ('data2_d' not in locals()):
        data_d = data1_d.copy()
    elif ('data2_d' in locals()) and ('data1_d' not in locals()): 
        data_d = data2_d.copy()

    # corregir columnas
    cols = []
    for col in data_d.columns:
        if col[-8:] == 'XACQRIO1':
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('caudal_m³/s')
        elif col[-8:] == 'XAINRIO1':
            data_d[col] = [round(d, 3) for d in data_d[col]]
            cols.append('nivel_m')
        elif col[-8:] == 'XAIPCINC':
            data_d[col] = data_d[col] * 24 * 60 / 5 # precipitación diaria acumulada
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('precipitacion_mm')
        elif col[-8:] == 'XAITEMEX':
            data_d[col] = [round(d, 1) for d in data_d[col]]
            cols.append('temperatura_ºC')
    data_d.columns = cols
    data_d.index.name = 'Fecha'

    # comprobar fechas
    st, en = data_d.index[0], data_d.index[-1]
    if data_d.shape[0] != pd.date_range(st, en).shape[0]:
        print('¡ERROR AL UNIR SERIES! Distinto número de filas que días entre la fecha inicial y final.')
        return
    
    # exportar
    if not os.path.exists(rutaSAIH + agregacion + '/'):
            os.makedirs(rutaSAIH + agregacion + '/')
    data_d.to_csv(rutaSAIH + agregacion + '/' + str(estacion) + '.csv', sep=',', na_rep=-100)

    if verbose == True:
        print('nº de días de las serie:\t', data_d.shape[0])
        print('variables:\t', list(data_d.columns))

In [ ]:
SAIH_CHC_diario2('A050', rutaSAIH, verbose=True)

In [ ]:
for i, stn in enumerate(stns):
    print('estación', str(stn), '\tserie', i, 'de', len(stns))
    SAIH_CHC_diario2(stn, rutaSAIH, verbose=True)
    print()

In [ ]:
def SAIH_CHC_diario3(estacion, ruta, freq=None, verbose=True):
    """Genera las series diarias para las estaciones del SAIH Cantábrico.
    
    Entradas:
    ---------
    estacion:  str o int. Nombre de la estación
    ruta:      str.
    freq:      str. Resolución temporal a la que agregar los datos: '1A' anual, '1M' mensual, 1D' diaria, '1H' horaria, 'None' datos originales cincominutales
    verbose:   boolean.
    
    Salidas:
    --------
    """

    # rutas
    rutaSAIH1 = ruta + '/Hasta junio de 2015/'
    rutaSAIH2 = ruta + '/Desde julio de 2015/'

    signames = ['AIPCINC', 'ACQRIO1', 'AINRIO1', 'AITEMEX']
    
    # PARTE 1
    # -------
    # encontrar archivos de la estación
    folders = os.listdir(rutaSAIH1)
    for f, folder in enumerate(folders):
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH1 + folder + '/'
            files = []
            for file in os.listdir(ruta):
                if file[:4] == folder[:4]:
                    files.append(file)

            # Importar datos cincominutales
            data1 = pd.DataFrame()
            for i, file in enumerate(files):
                # importar serie original
                aux = pd.read_csv(ruta + file, sep=';', encoding='latin-1', decimal=',', low_memory=False)
                aux.dropna(axis=0, how='all', inplace=True)
                aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
                aux.set_index('Fecha', drop=True, inplace=True)
                # reordenar 'aux' por señales
                signals, cols = [], []
                for signal in aux['Nombre señal'].unique():
                    if signal[-7:] in signames:
                        signals.append(signal)
                aux2 = pd.DataFrame(columns=cols)
                for signal in signals:
                    temp = aux.loc[aux['Nombre señal'] == signal, 'Valor']
                    cols = list(aux2.columns)
                    aux2 = pd.concat((aux2, temp), axis=1)
                    aux2.columns = cols + [str(estacion) + 'X' + signal[-7:]]
                # concatenar a la serie generada
                data1 = pd.concat((data1, aux2), axis=0)
            del files, folders
            break

    # PARTE 2
    # -------
    # encontrar carpeta de la estación
    folders = os.listdir(rutaSAIH2)
    for folder in folders:
        if folder[-4:] == str(estacion):
            ruta = rutaSAIH2 + folder + '/'
            # encontrar archivos de la estación
            files = os.listdir(ruta)

            # Importar datos cincominutales
            data2 = pd.DataFrame()
            for file in files:
                aux = pd.read_csv(ruta + file, sep=';', decimal=',', encoding='latin-1', skiprows=1)
                aux['Fecha/Hora'] = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux['Fecha/Hora']]
                aux.set_index('Fecha/Hora', drop=True, inplace=True)
                aux.index.name = 'Fecha'
                cols = []
                for col in aux.columns:
                    if col[-7:] in signames:
                        cols.append(col)
                aux = aux.loc[:, cols]
                data2 = pd.concat((data2, aux), axis=0)
            del files, folders   
            break

    # UNIR SERIES
    # -----------
    # unir las dos series
    if ('data1' in locals()) and ('data2' in locals()):
        st1, en1 = data1.index[0], data1.index[-1]
        st2, en2 = data2.index[0], data2.index[-1]
        idx = pd.date_range(st1, en2)
        data = pd.DataFrame(index=idx, columns=data1_d.columns)
        data.loc[st2:en2,:] = data2_d
        data.loc[st1:en1,:] = data1_d
    elif ('data1' in locals()) and ('data2' not in locals()):
        data = data1.copy()
    elif ('data2' in locals()) and ('data1' not in locals()): 
        data = data2.copy()
    
    # Agregar datos
    if freq == None:
        freq = '5min'
        data_ag = data.copy()
    elif freq == '1H':
        by = [data.index.year, data.index.month, data.index.day, data.index.hour]
        data_ag = data.groupby(by=by).mean()
        data_ag.index = [pd.datetime(y, m, d, h) for (y, m, d, h) in data_ag.index]
    elif freq == '1D':
        by = [data.index.year, data.index.month, data.index.day]
        data_ag = data.groupby(by=by).mean()
        data_ag.index = [pd.datetime(y, m, d).date() for (y, m, d) in data_ag.index]
    elif freq == '1M':
        by = [data.index.year, data.index.month]
        data_ag = data.groupby(by=by).mean()
        data_ag.index = [pd.datetime(y, m).date() for (y, m) in data_ag.index]
    elif freq == '1A':
        by = [data.index.year,]
        data_ag = data.groupby(by=by).mean()
        data_ag.index = [pd.datetime(y).date() for (y) in data_ag.index]
    
    st, en = data1_d.index[0], data1_d.index[-1]
    if data_ag.shape[0] != pd.date_range(st, en, freq=freq).shape[0]:
        aux = data_ag.copy()
        data1_ag = pd.DataFrame(index=pd.date_range(st, en, freq=freq), columns=data_ag.columns)
        for date in aux.index:
            data_ag.loc[date, :] = aux.loc[date, :]

    # corregir columnas
    cols = []
    for col in data_ag.columns:
        if col[-8:] == 'XACQRIO1':
            data_ag[col] = [round(d, 1) for d in data_ag[col]]
            cols.append('caudal_m³/s')
        elif col[-8:] == 'XAINRIO1':
            data_ag[col] = [round(d, 3) for d in data_ag[col]]
            cols.append('nivel_m')
        elif col[-8:] == 'XAIPCINC':
            data_ag[col] = data_ag[col] * 24 * 60 / 5 # precipitación diaria acumulada
            data_ag[col] = [round(d, 1) for d in data_ag[col]]
            cols.append('precipitacion_mm')
        elif col[-8:] == 'XAITEMEX':
            data_ag[col] = [round(d, 1) for d in data_ag[col]]
            cols.append('temperatura_ºC')
    data_ag.columns = cols
    data_ag.index.name = 'Fecha'

    # comprobar fechas
    st, en = data_ag.index[0], data_ag.index[-1]
    if data_ag.shape[0] != pd.date_range(st, en, freq=freq).shape[0]:
        print('¡ERROR AL UNIR SERIES! Distinto número de filas que días entre la fecha inicial y final.')
        return
    
    # exportar
    if not os.path.exists(ruta + freq + '/'):
            os.makedirs(ruta + freq + '/')
    data_ag.to_csv(ruta + freq + '/' + str(estacion) + '.csv', sep=',', na_rep=-100)

    if verbose == True:
        print('nº de días de las serie:\t', data_ag.shape[0])
        print('variables:\t', list(data_ag.columns))

In [ ]:
SAIH_CHC_diario3('A047', rutaSAIH, freq=None, verbose=True)

In [ ]:
aux = pd.read_csv(rutaSAIH1 + 'A047/A047-2007.csv', sep=';', decimal=',', encoding='latin-1', skiprows=0)
aux.dropna(axis=0, how='all', inplace=True)
aux.Fecha = [datetime.strptime(date, '%d/%m/%Y %H:%M') for date in aux.Fecha]
aux.set_index('Fecha', drop=True, inplace=True)

In [ ]:
aux.head()

In [ ]:
aux.index[0], aux.index[-1]

In [ ]:
rango = pd.date_range(aux.index[0], aux.index[-1], freq='A')
rango

In [ ]:
aux.index.hour